In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate
from scipy import integrate as integrate
import pandas as pd
from scipy.optimize import minimize
from scipy.integrate import odeint
from scipy import integrate as integrate
from scipy.interpolate import CubicSpline
from scipy.interpolate import lagrange
from scipy import interpolate

In [2]:
c = 2.99792458e+5
omega_v0 = 0.0014
omega_bh2 = 0.02221
G = 6.67408*10**(-11)

In [3]:
np.pi

3.141592653589793

In [4]:
def kappa(parameters):
    omega_m0,omega_k0,alpha,Ho = parameters
    return (8/3)*((alpha + 4)/(alpha + 2))*(2/3 * alpha*(alpha + 2))**(alpha/2)

In [5]:
def V(parameters,phi):
    alpha = parameters[2]
    return 0.5*k(parameters)*phi**(-alpha)/(8*np.pi * G)

In [6]:
def KG_eqn(w, t,parameters):
    
    phi,phi_dot, a = w
    omega_m0,omega_k0,alpha,Ho = parameters
    k_val = kappa(parameters)
    a_dot = a*np.sqrt((omega_k0 / a**2 + omega_m0 / a**3)*Ho**2  + 8*np.pi*G / 3 *(0.5*phi_dot**2 + V(parameters,phi)))
    f = [phi_dot,
         V(parameters,phi)*alpha/phi - 3/a * phi_dot* a_dot,
         a_dot]
    return f

In [7]:
def y0(alpha):
    t0 = 10**(-5)
    a_initial = (t0**(2./3.))
    phi_initial =   ((2.*alpha*(alpha+2.)/3.)**(1/2.))*((t0)**(2./(alpha+2.)))
   
    phi_dot_initial = (((8./3.)*alpha*(1./(alpha + 2.)))**(1./2.))/(t0)**(alpha/(alpha + 2.))
    
    Ho = 70 
    rho_cr0 = 3*Ho**2/(8*np.pi)
    
    return phi_initial,phi_dot_initial,phi_initial

In [8]:
from scipy.integrate import odeint

def Omega_phi(parameters,z):
    omega_m0,omega_k0,alpha,Ho = parameters
    t_initial = 10**(-5)
    t_final = 150
    dt = 10**(-4)
    t = np.arange(0., t_final, dt)
    sol = odeint (KG_eqn, y0(alpha), t, args=(parameters,))
    a=sol[:,0]
    phi=sol[:,1]
    phi_dot = sol[:,2]
    rho_m = 4.0/(9.0*(a**3))
    kappa_alpha = kappa(parameters)
    V = (1/2)*kappa_alpha/(phi)**(alpha)
    rho_phi = (1/6)*(0.5*phi_dot**2 + V)
    phi=sol[:,0]
    phi_dot=sol[:,1]
    a = sol[:,2]
    kappa_alpha = kappa(parameters)
    V = 0.5*k(parameters)*phi**(-alpha)/(8*np.pi * G)
    rho_phi = (1/6)*(0.5*phi_dot**2 + V)
    omega_phi = 8*np.pi*G / (3*Ho**2) * rho_phi
    z_calc = 1 / a - 1
    g = interpolate.interp1d(z_calc,omega_phi,fill_value = "extrapolate")
    return g(z)

In [9]:
def E(z, parameters):   
    omega_m0,omega_k0,alpha,h0 = parameters
    h=h0/100
    #omega_cb0 = omega_m0 - omega_v0
    #omega=(omega_bh2+omega_ch2+omega_vh2)/h**2
    return ((omega_m0*((1+z)**3)) + omega_k0*(1+z)**2 + Omega_phi(parameters,z))**(1/2)
def f(z,parameters):
    return 1/E(z,parameters)

In [10]:
def Chi_sq1(parameters,H_obs,z_obs,sigma_obs):
    n = z_obs.shape[0]
    H_theo = np.zeros(n)
    omega_m0,omega_k0,alpha,Ho = parameters
    chi_sq=0
    for i in range(n):
        H_theo[i]= Ho*(E(z_obs[i],parameters))
        chi_sq= chi_sq+(H_obs[i]-H_theo[i])**2/(sigma_obs[i])**2
    return chi_sq

In [11]:
def dH(parameters,z):
    omega_m0,omega_k0,alpha,Ho = parameters
    return c/(Ho*E(z,parameters))
def dC(parameters,z):
    omega_m0,omega_k0,alpha,Ho = parameters
    return (c/Ho)*integrate.quad(f,0,(z),args=(parameters,))[0]
def dM(parameters,z):
    omega_m0,omega_k0,alpha,Ho = parameters
    if omega_k0 == 0:
        return dC(parameters,z)
    elif omega_k0 > 0:
        return (c / (Ho*np.sqrt(omega_k0)))*np.sinh(np.sqrt(omega_k0)*Ho*dC(parameters,z) / c)
    elif omega_k0 < 0:
        mod_omega_k0 = -omega_k0
        return (c / (Ho*np.sqrt(mod_omega_k0)))*np.sin(np.sqrt(mod_omega_k0)*Ho*dC(parameters,z) / c)
def dA(parameters,z):
    return dM(parameters,z)/(1+z)
def dV(parameters,z):
    Ho = parameters[3]
    return (((c/Ho)*((dM(parameters,z))**2)*(z/E(z,parameters)))**(1/3))

In [12]:
def rs(parameters,z):
    omega_m0,omega_k0,alpha,Ho = parameters
    h = Ho/100
    num =np.exp(-72.3*(omega_v0*h**2+0.0006)**2)
    den = ((omega_bh2)**(0.12807))*(((omega_m0 - omega_v0)*h**2)**(0.25351))
    return 55.154*num/den

In [14]:
parameters = [0.321,-0.126,0.938,65.93]

In [15]:
dV(parameters,0.38)*147.5/rs(parameters,0.38)

NameError: name 'k' is not defined

In [16]:
def Chi_sq2(parameters,D_obs,z_obs,cov_mat):
    datapoints= z_obs.shape
    D_theo= np.zeros(datapoints)
    Ho=parameters[3]
    
    dm_indices= [0,2,4]
    h_indices=[3,1,5]
    Dv_indices = [6,8]
    dA_indices = [7]
    dhrs_indices = [9]
    dmrs_indices = [10]
    
    rs_fid= 147.78
    
    for i in Dv_indices:
        if i==6:
            D_theo[i]= dV(parameters,z_obs[i])*147.5/rs(parameters,z_obs[i])
        else:
            D_theo[i]= dV(parameters,z_obs[i])*rs_fid/rs(parameters,z_obs[i])
    for i in h_indices:
        D_theo[i]=Ho*E(z_obs[i],parameters)*rs(parameters,z_obs[i])/rs_fid
    for i in dm_indices:
        D_theo[i]=dM(parameters,z_obs[i])*rs_fid/rs(parameters,z_obs[i])
    for i in dhrs_indices:
        D_theo[i]=(dH(parameters,z_obs[i]))/ rs(parameters,z_obs[i])
    for i in dmrs_indices:
        D_theo[i]=(dM(parameters,z_obs[i]))/ rs(parameters,z_obs[i])
    for i in dA_indices:
        D_theo[i]=(dA(parameters,z_obs[i]))/rs(parameters,z_obs[i])
    #print(parameters)
    #print("D_theo1",D_theo)
    #print("D_obs",D_obs)
    D_theo= D_theo-D_obs
    #print("D_theo",D_theo)
    A = np.matmul(np.linalg.inv(cov_mat),D_theo)
    #print("A",A)
    final = (np.matmul(D_theo.T,A))
    #print(final)
    #print(np.linalg.inv(cov_mat))
    #print(final)
    return final[0,0]

In [17]:
def Chi_sq(parameters,H_obs1,z_obs1,sigma_obs1,D_obs,z_obs2,cov_matrix):
    #print(Chi_sq3(parameters,H_obs1,z_obs1,sigma_obs1))
    return (Chi_sq2(parameters,D_obs,z_obs2,cov_matrix)+ Chi_sq1(parameters,H_obs1,z_obs1,sigma_obs1))

In [18]:
def lnprior1(parameters):
    omega_m0,omega_k0,wX,Ho = parameters
    omega_lambda = 1-omega_k0 - omega_m0
    if 0.1<=omega_m0<=0.7 and 50<=Ho<=85 and -2<=wX<=0 and -0.7<=omega_k0<=0.7:
        return 0
    return -np.inf

In [19]:
def log_likelihood(parameters,H_obs1,z_obs1,sigma_obs1,D_obs,z_obs,cov_matrix):
    return -Chi_sq(parameters,H_obs1,z_obs1,sigma_obs1,D_obs,z_obs,cov_matrix)/2

In [20]:
def log_probability(parameters,H_obs1,z_obs1,sigma_obs1,D_obs,z_obs,cov_matrix):
    lp = lnprior1(parameters)
    if not np.isfinite(lp):
        return -np.inf
    #print(lp)
    #print(lp + log_likelihood(parameters,H_obs1,z_obs1,sigma_obs1,D_obs,z_obs,cov_matrix))
    return lp + log_likelihood(parameters,H_obs1,z_obs1,sigma_obs1,D_obs,z_obs,cov_matrix)

In [21]:
Data = pd.read_csv("H(z) data - Sheet1.csv")
z_obs1 = Data['z']
z_obs1 = z_obs1.to_numpy()
z_obs1 = np.reshape(z_obs1,(31,1))
H_obs1 = Data['H(z)']
sigma_obs1 = Data['sigma']
print(Data)
print(z_obs1.shape)

         z   H(z)  sigma
0   0.0700   69.0   19.6
1   0.0900   69.0   12.0
2   0.1200   68.6   26.2
3   0.1700   83.0    8.0
4   0.1790   75.0    4.0
5   0.1990   75.0    5.0
6   0.2000   72.9   29.6
7   0.2700   77.0   14.0
8   0.2800   88.8   36.6
9   0.3520   83.0   14.0
10  0.3802   83.0   13.5
11  0.4000   95.0   17.0
12  0.4004   77.0   10.2
13  0.4247   87.1   11.2
14  0.4497   92.8   12.9
15  0.4700   89.0   50.0
16  0.4783   80.9    9.0
17  0.4800   97.0   62.0
18  0.5930  104.0   13.0
19  0.6800   92.0    8.0
20  0.7810  105.0   12.0
21  0.8750  125.0   17.0
22  0.8800   90.0   40.0
23  0.9000  117.0   23.0
24  1.0370  154.0   20.0
25  1.3000  168.0   17.0
26  1.3630  160.0   33.6
27  1.4300  177.0   18.0
28  1.5300  140.0   14.0
29  1.7500  202.0   40.0
30  1.9650  186.5   50.4
(31, 1)


In [22]:
Data = pd.read_csv("BAO data - Sheet1 (3).csv")
print(Data)
Data['value'][9] = 8.86
Data['value'][10] = 37.41
Data['z'][9] = 2.34
Data['z'][10] = 2.34
#print(Data)

        z      value   sigma
0   0.380  1512.3900     NaN
1   0.380    81.2087     NaN
2   0.510  1975.2200     NaN
3   0.510    90.9029     NaN
4   0.610  2306.6800     NaN
5   0.610    98.9647     NaN
6   0.122   539.0000   17.00
7   0.810    10.7500    0.43
8   1.520  3843.0000  147.00
9   2.334    37.5000     NaN
10  2.334     8.9900     NaN


In [23]:
z_obs = (Data['z'].to_numpy()).astype(float)
z_obs = np.reshape(z_obs, (11,1))
Data = Data.to_numpy()
D_obs = Data[:,1].astype(float)
D_obs = np.reshape(D_obs,(11,1))
print(z_obs.shape)
print(D_obs)

(11, 1)
[[1512.39  ]
 [  81.2087]
 [1975.22  ]
 [  90.9029]
 [2306.68  ]
 [  98.9647]
 [ 539.    ]
 [  10.75  ]
 [3843.    ]
 [   8.86  ]
 [  37.41  ]]


In [24]:
cov_matrix2 = np.matrix([[624.707, 23.729, 325.332, 8.34963, 157.386, 3.57778,0,0,0,0,0],
                    [23.729, 5.60873, 11.6429, 2.33996, 6.39263, 0.968056,0,0,0,0,0],
                    [325.332, 11.6429, 905.777, 29.3392, 515.271, 14.1013,0,0,0,0,0],
                    [8.34963, 2.33996, 29.3392, 5.42327, 16.1422, 2.85334,0,0,0,0,0],
                    [157.386, 6.39263, 515.271, 16.1422, 1375.12, 40.4327,0,0,0,0,0],
                    [3.57778, 0.968056, 14.1013, 2.85334, 40.4327, 6.25936,0,0,0,0,0],
                    [0,0,0,0,0,0,17**2,0,0,0,0],
                     [0,0,0,0,0,0,0,0.43**2,0,0,0],
                     [0,0,0,0,0,0,0,0,147**2,0,0],
                     [0,0,0,0,0,0,0,0,0,0.0841,-0.183396],
                     [0,0,0,0,0,0,0,0,0,-0.183396,3.4596]])

In [25]:
initial = np.array([0.3,0.1,0.1,70]) 
soln = minimize(Chi_sq, initial, args=(H_obs1,z_obs1,sigma_obs1,D_obs,z_obs,cov_matrix2),bounds=[(-1,1),(-1,1),(0,np.inf),(0,np.inf)])
print(soln)

NameError: name 'k' is not defined

In [7]:
def vectorfield1(w,t,parameters):
    phi,phi_dot,a = w #I used p for phi, v for d(phi)/dt, and a for the
    ##scale factor.
    al, k, m, K = parameters #These are the parameters of the model. See below.
    f = [phi_dot,
    -3.*phi_dot*(((4./(9.*a**3.))) + (1./12.)*(phi_dot**2. + (k*m)/(phi**(al)))
    - K/(a**2.))**(1./2.)
    + ((k*al*m)/2.)/(phi**(al + 1.)),
    (((4./9.)/a) + (((a**2.)/12.))*(phi_dot**2.
    + (k*m)*(phi**(-al))) - K)**(1./2.)]
    return f


In [ ]:
def KG_eqn(y, t, parameters):
    #a,phi,phi_dot,rho_m = y
    a,phi,phi_dot = y
    rho_m = 4.0/(9.0*a**3)
    alpha=parameters[0]
    kappa_alpha = kappa(para)
    mp = 1
    V = (1/2)*kappa_alpha*mp**2/(phi)**(alpha)
    rho_phi = (1/6)*(0.5*phi_dot**2 + V)
    Hubble_rate = np.sqrt( (rho_phi + rho_m))
    #dydt = [a*Hubble_rate,phi_dot,-3*Hubble_rate*phi_dot+ 0.5*kappa_alpha*alpha/(phi**(alpha+1)), -3*rho_m*Hubble_rate ]
    dydt = [a*Hubble_rate,phi_dot,-3*Hubble_rate*phi_dot+ 0.5*kappa_alpha*alpha/(phi**(alpha+1)) ]
    return dydt

In [ ]:
def y0(alpha):
    t0 = 10**(-5)
    a_initial = (t0**(2./3.))
    phi_initial =   ((2.*alpha*(alpha+2.)/3.)**(1/2.))*((t0)**(2./(alpha+2.)))
   
    phi_dot_initial = (((8./3.)*alpha*(1./(alpha + 2.)))**(1./2.))/(t0)**(alpha/(alpha + 2.))
    
    Ho = 70 # Hubble parameter today in units of 100 km/s/Mpc (Planck 2015)
    rho_cr0 = 3*Ho**2/(8*np.pi)
    #h = 0.678 # Hubble parameter today in units of 100 km/s/Mpc (Planck 2015)
    #rho_cr0 = 8.0992*(h**2)*10**(-47) / (2.435*10**18)**4
    # calculate the matter density today
    
    #Omega_m = 0.28
    #rho_m0 = Omega_m*rho_cr0
    #rho_initial = rho_m0/(a_initial**3) # initial value for the matter density
    
    return a_initial,phi_initial,phi_dot_initial

In [ ]:
def E_zobs(parameters,z_obs):
    alpha= parameters[0]
    omega_bh2=parameters[1]
    omega_ch2=parameters[2]
    h=parameters[3]/100
    omega_vh2=0.06/93.14
    omega=(omega_bh2+omega_ch2+omega_vh2)/h**2
    #print(omega)
    t_initial = 10**(-5)
    t_final = 150
    dt = 10**(-4)
    t = np.arange(0., t_final, dt)
    
    #print(y0(alpha))
    sol = odeint (KG_eqn, y0(alpha), t, args=(parameters,))
    
    a=sol[:,0]
    phi=sol[:,1]
    phi_dot = sol[:,2]
    rho_m = 4.0/(9.0*(a**3))
    kappa_alpha = kappa(alpha)
    V = (1/2)*kappa_alpha/(phi)**(alpha)
    rho_phi = (1/6)*(0.5*phi_dot**2 + V)
    omega_exp = rho_m/(rho_phi+rho_m)
    #i = np.where(omega_exp>=omega)
    #i=i[0]
    #if (len(i)>1):
        #i=i[-1]
    #else:
        #i=len(t)-1
    #print(i)
    omega_exp = np.flipud(omega_exp)
    omega_exp_diff = omega_exp[1:]
    omega_exp_diff = np.diff(omega_exp)
    zeros= np.where(omega_exp_diff>0)
    zeros=zeros[0]
    #print(zeros)
    zeros=zeros+1
    omega_exp_new= omega_exp[zeros]
    aflip = np.flipud(a[zeros])
    g = interpolate.interp1d(omega_exp_new,aflip,fill_value = "extrapolate")
    #size = omega_exp.size
    a1 = g(omega)
    #print(a1)
    rm = interpolate.interp1d(a,rho_m,fill_value = "extrapolate")
    rp =interpolate.interp1d(a,rho_phi,fill_value = "extrapolate")
    #rho_critical = rho_m[i]+rho_phi[i]
    rho_critical = rm(a1)+rp(a1)
    Einv_array = np.zeros(len(z_obs))
    for i in range(len(z_obs)):
        Einv_array[i] = integrate.quad(Einv,0,(z_obs[i]),args=(rm,rp,rho_critical,a1,))[0]
    #print(rho_critical)
    #a = a/sol[i,0]
#     z_values = (1/a)-1
#     omega_m_theo=rho_m/rho_critical
#     omega_phi_theo=rho_phi/rho_critical
#     start = time.time()
#     E_values = np.sqrt(omega_m_theo+omega_phi_theo)
#     z_values = np.flipud(z_values)
#     E_values = np.flipud(E_values)
#     z_bw_0_1000 = np.where((z_values>=0) & (z_values<=20))
#     z_bw_0_1000_values =z_values[z_bw_0_1000]
#     E_bw_0_1000_values= E_values[z_bw_0_1000]
#     #print(z_bw_0_1000_values)
#     f = CubicSpline(z_bw_0_1000_values, E_bw_0_1000_values)
    a_obs = 1/(1+z_obs)
    a_obs = a_obs*a1
    omega_m_obs = rm(a_obs)/rho_critical
    omega_phi_obs = rp(a_obs)/rho_critical
    E_array = np.sqrt(omega_m_obs+omega_phi_obs)
    return (E_array, Einv_array)
